# 13. Redes neuronales recurrentes
[**Python Deep Learning** Introducción práctica con Keras y TensorFlow 2. Jordi Torres. Editorial Marcombo ISBN: 9788426728289 ](https://www.marcombo.com/python-deep-learning-9788426728289/)

### Importar TensorFlow 2.0  y otras librerias

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

### Descarga del conjunto de datos

In [ ]:

from google.colab import files
#se debe cargar el fichero “Libro-Deep-Learning-introduccion-practica-con-Keras-1a-parte.txt”
files.upload()

path_to_fileDL ='/content/noticias.txt'

#path_to_fileDL = tf.keras.utils.get_file('Shakespear.txt', 'https://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt')


Saving noticias.txt to noticias.txt


In [ ]:


text = open(path_to_fileDL, 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(text)))

vocab = sorted(set(text))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        239264 carácteres
El texto está compuesto de estos 110 carácteres:
['\n', ' ', '!', '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '¡', 'ª', '«', 'º', '»', '¿', 'À', 'Á', 'É', 'Í', 'Ó', 'Ú', 'à', 'á', 'ç', 'è', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '“', '”', '…']


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '%' :   5,
  '&' :   6,
  "'" :   7,
  '(' :   8,
  ')' :   9,
  '*' :  10,
  '+' :  11,
  ',' :  12,
  '-' :  13,
  '.' :  14,
  '/' :  15,
  '0' :  16,
  '1' :  17,
  '2' :  18,
  '3' :  19,
  '4' :  20,
  '5' :  21,
  '6' :  22,
  '7' :  23,
  '8' :  24,
  '9' :  25,
  ':' :  26,
  ';' :  27,
  '?' :  28,
  '@' :  29,
  'A' :  30,
  'B' :  31,
  'C' :  32,
  'D' :  33,
  'E' :  34,
  'F' :  35,
  'G' :  36,
  'H' :  37,
  'I' :  38,
  'J' :  39,
  'K' :  40,
  'L' :  41,
  'M' :  42,
  'N' :  43,
  'O' :  44,
  'P' :  45,
  'Q' :  46,
  'R' :  47,
  'S' :  48,
  'T' :  49,
  'U' :  50,
  'V' :  51,
  'W' :  52,
  'X' :  53,
  'Y' :  54,
  'Z' :  55,
  '[' :  56,
  ']' :  57,
  'a' :  58,
  'b' :  59,
  'c' :  60,
  'd' :  61,
  'e' :  62,
  'f' :  63,
  'g' :  64,
  'h' :  65,
  'i' :  66,
  'j' :  67,
  'k' :  68,
  'l' :  69,
  'm' :  70,
  'n' :  71,
  'o' :  72,
  'p' :  73,
  'q' :  74,
  'r' :  75,
  's' :  76,

In [ ]:
text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
print ('texto: {}'.format(repr(text[:50])))
print ('{}'.format(repr(text_as_int[:50])))

texto: 'text\n" Algunas de las voces extremistas más conoci'
array([77, 62, 81, 77,  0,  3,  1, 30, 69, 64, 78, 71, 58, 76,  1, 61, 62,
        1, 69, 58, 76,  1, 79, 72, 60, 62, 76,  1, 62, 81, 77, 75, 62, 70,
       66, 76, 77, 58, 76,  1, 70, 98, 76,  1, 60, 72, 71, 72, 60, 66])


### Preparar los datos para entrenar la RNN


In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

seq_length = 100
 
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)



In [ ]:
for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

'text\n" Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento '
'de la conocida como alt-right o ligadas a grupos antisemitas o xenófobos , han sido eliminadas de un '
'plumazo de la redes sociales Facebook e Instagram. Se trat"\n" Después de casi dos años y medio de luc'
'has políticas y judiciales, Donald Trump ha presentado hoy su proyecto de inmigración para Estados Un'
'idos . Es un proyecto elaborado por el yerno y asesor de Donald Trump, Jared Kushner, que sólo concre'
'ta una cosa:"\n Dos periodistas birmanos de la agencia Reuters fueron liberados este martes al recibir'
' un perdón del presidente de su país tras pasar año y medio en prisión por revelar una matanza del Ej'
'ército contra la minoría musulmana rohingya. Wa Lone y Kyaw Soe Oo\n El Cuerpo Nacional de Policía ha '
'detenido a cuatro personas acusadas de formar parte de un grupo criminal dedicado al tráfico de seres'
' humanos en la frontera francoespañola de Irún (Gipuzkoa). 

In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'text\n" Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento'
Target data: 'ext\n" Algunas de las voces extremistas más conocidas de EE.UU., cuentas asociadas con al movimiento '


In [ ]:
print (dataset)

<MapDataset element_spec=(TensorSpec(shape=(100,), dtype=tf.int64, name=None), TensorSpec(shape=(100,), dtype=tf.int64, name=None))>


In [ ]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print (dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


### Construcción del modelo

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  model.add(Embedding(input_dim=vocab_size, 
                      output_dim=embedding_dim, 
                      batch_input_shape=[batch_size, None]))
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(vocab_size))
  return model

In [ ]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           15872     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 62)            63550     
                                                                 
Total params: 5,326,398
Trainable params: 5,326,398
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")


Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):  
    example_batch_predictions = model(input_example_batch)
    print("Prediction: ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction:  (64, 100, 110) # (batch_size, sequence_length, vocab_size)


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices_characters

array([ 74,  62, 109,  14,  99,  76,  26,  27,  56,  61,  55,  71,  75,
       103,  98,  96,  38, 103,  64,  20,  25,  82, 103,  73, 105,  32,
         3,  73,  21,  27,   6,  55,  64,  10,  25,  51,  36,   1,  64,
        85,  99,  83,  83,  83,  81,   7,  45,  26,   6,  96,  89,  79,
        63, 108, 102,  49, 107,  77,  12,  81,  36,  94,  66,  49,  76,
         0,  44,  10,   3,  39,  39,  55, 104,   1,  70,  73,  33,  52,
       105,  62,  72,  58,  94,   0,  56, 102,  45,  18,   4,   7,  33,
         0,  97,  43,  74,  26,  67, 103,  22,  10])

Entrenar el modelo

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

Configurar el *checkpoints*

In [ ]:
 # directorio
checkpoint_dir = './training_checkpoints'
# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

*Training*

In [ ]:
EPOCHS=70
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/70
37/37 [==============================] - 4s 73ms/step - loss: 3.2097
Epoch 2/70
37/37 [==============================] - 3s 68ms/step - loss: 2.7213
Epoch 3/70
37/37 [==============================] - 3s 67ms/step - loss: 2.4812
Epoch 4/70
37/37 [==============================] - 3s 68ms/step - loss: 2.3491
Epoch 5/70
37/37 [==============================] - 3s 70ms/step - loss: 2.2577
Epoch 6/70
37/37 [==============================] - 3s 75ms/step - loss: 2.1874
Epoch 7/70
37/37 [==============================] - 3s 69ms/step - loss: 2.1173
Epoch 8/70
37/37 [==============================] - 3s 70ms/step - loss: 2.0504
Epoch 9/70
37/37 [==============================] - 3s 71ms/step - loss: 1.9923
Epoch 10/70
37/37 [==============================] - 3s 71ms/step - loss: 1.9359
Epoch 11/70
37/37 [==============================] - 3s 70ms/step - loss: 1.8806
Epoch 12/70
37/37 [==============================] - 3s 70ms/step - loss: 1.8284
Epoch 13/70
37/37 [==================

### Generación de texto

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_70'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):

  num_generate = 300
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  temperature = 0.5

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"la"))

la solo confirmar una media de 23.120 cotizantes extranjeros en el ejército en un comunicado, ha instado a que todos los"
La Comunidad Valenciana pasa a alerta amarilla espacio político provisional, como el pueblo con su posible concejales del Estado Isale, el capital de Dia, el consejero delegado de 
